In [13]:
import numpy as np
import sys
# def Logistic(kwargs["x"]=None, kwargs['alpha']=None, kwargs['beta']=None, kwargs['gamma']=None, kwargs['_lambda']=None):
def Logistic(**kwargs):
  
    #
    # p = Logistic(kwargs["x"], kwargs['alpha'], kwargs['beta'], kwargs['gamma'], lambda)
    #
    # The logistic psychometric function
    #
    # - kwargs["X"]:  level of the stimulus;
    # - kwargs['ALPHA']: midpoint of the psychometric function. It is the value
    # of the stimulus where the function crosses the mean probability between
    # kwargs['gamma'] and lambda (i.e., .75 for a 2AFC, or .66 for a 3AFC, etc.);
    # - kwargs['BETA']: slope of the function. It is the rate of change in the
    # subject's performance with stimulus level;
    # - kwargs['GAMMA']: lower limit of the psychometric function ekwargs["x"]pressed as a
    # proportion. It ranges from 0 to 1. The meaning of kwargs['gamma'] is different for
    # yes/no and nAFC tasks. In nAFC tasks kwargs['gamma'] corresponds to chance level
    # (i.e., 0.5 in a 2AFC task). In yes/no tasks kwargs['gamma'] corresponds to the
    # false alarm rate of the subject;
    # - LAMBDA: the subject's lapse rate ekwargs["x"]pressed as a proportion.

    #ask

    if len(sys.argv) < 5:
        kwargs['_lambda'] = 0
    

    P = kwargs['gamma'] + ((1 - kwargs['_lambda'] - kwargs['gamma']) * (1 / (1 + np.exp(kwargs['beta'] * (kwargs['alpha'] - kwargs["x"])))))

    return P



#def InvLogistic(p_target=None, kwargs['alpha']=None, kwargs['beta']=None, kwargs['gamma']=None, kwargs['_lambda']=None):
def InvLogistic(**kwargs):

    #
    # kwargs["x"] = InvLogistic(p_target, kwargs['alpha'], kwargs['beta'], kwargs['gamma'], lambda)
    #
    # The inverse of the logistic psychometric function
    #
    # - P_TARGET: the performance we are tracking ekwargs["x"]pressed as a proportion;
    # - kwargs['ALPHA']: the midpoint of the psychometric function. It is the value
    # of the stimulus where the function crosses the mean probability between
    # kwargs['gamma'] and lambda (i.e., .75 for a 2AFC, or .66 for a 3AFC, etc.);
    # - kwargs['BETA']: the slope of the function. It is the rate of change in the
    # subject's performance with stimulus level;
    # - kwargs['GAMMA']: the lower limit of the psychometric function ekwargs["x"]pressed as a
    # proportion. It ranges from 0 to 1. The meaning of kwargs['gamma'] is different for
    # yes/no and nAFC tasks. In nAFC tasks kwargs['gamma'] corresponds to chance level
    # (i.e., 0.5 in a 2AFC task). In yes/no tasks kwargs['gamma'] corresponds to the
    # subject's false alarm rate;
    # - LAMBDA: the subject's lapse rate ekwargs["x"]pressed as a proportion.

    if len(sys.argv)< 5:
        kwargs['_lambda'] = 0
    

    kwargs["x"] = kwargs['alpha'] - (1 / kwargs['beta']) * np.log(((1 - kwargs['_lambda'] - kwargs['gamma']) / (kwargs['p_target'] - kwargs['gamma'])) - 1)

    return kwargs["x"]
# @mfunction("ll")
# def CalculateLikelihood(kwargs["x"]=None, kwargs["responses"]=None, kwargs['alpha']=None, kwargs['beta']=None, kwargs['gamma']=None, kwargs['_lambda']=None):
def CalculateLikelihood(**kwargs):

    #
    # ll = CalculateLikelihood(kwargs["x"], kwargs["responses"], kwargs['alpha'], kwargs['beta'], kwargs['gamma'], lambda)
    #
    # This function returns the likelihood of a given psychometric function.
    #
    # - kwargs["X"]: an array containing the stimuli levels presented thus far;
    # - kwargs["RESPONSES"]: an array containing the subject's kwargs["responses"] collected thus
    # far. kwargs["Responses"] must be coded as "0" (wrong or "no"), % "1" (correct, or
    # "yes");
    # - kwargs['ALPHA']: the midpoint of the psychometric function;
    # - kwargs['BETA']: the slope of the psychometric function;
    # - kwargs['GAMMA']: the lower limit of the psychometric function ekwargs["x"]pressed as a
    # proportion;
    # - LAMBDA: the subject's lapse rate ekwargs["x"]pressed as a proportion.

    #warning(mstring('off'))

    ll = 0
    for i in range(1, len(kwargs["x"])):
        p = Logistic(x = kwargs["x"][i], alpha=kwargs['alpha'], beta=kwargs['beta'], gamma=kwargs['gamma'], _lambda=kwargs['_lambda'])
        if kwargs["responses"][i] == 1:
            ll = ll + np.log(p)
        else:
            ll = ll + np.log(1 - p)
        
    return ll


# def FindThreshold(p_target=None, kwargs["x"]=None, kwargs["responses"]=None, kwargs['alpha']=None, kwargs['beta']=None, kwargs['gamma']=None, kwargs['_lambda']=None):
def FindThreshold(**kwargs):

    #
    # [level, FA] = FindThreshold (p_target, kwargs["x"], kwargs["responses"], kwargs['alpha'], kwargs['beta'], kwargs['gamma'], lambda)
    #
    # This function first looks for the most likely psychometric function
    # within the range "firstpsyfun", "lastpsyfun" of the vector kwargs['alpha'].
    # Successively, it calculates the stimulus level for the nekwargs["x"]t trial at the
    # desired level of performance p_target, and returns this value to the user
    # together with an estimate of the subject's false alarm rate FA. If the
    # task is a nAFC, FA is equal to chance level. 
    #
    # - P_TARGET: the subject's performance we are targeting; it is the point
    # of the psychometric function we are tracking; 
    # - kwargs["X"]: an array containing the stimuli levels presented thus far;
    # - kwargs["RESPONSES"]: an array containing the subject's kwargs["responses"] collected thus
    # far. kwargs["Responses"] must be coded as "0" (wrong or "no"), "1" (correct, or
    # "yes");
    # - kwargs['ALPHA']: an array containing the midpoins of the psychometric functions
    # that will drive the threshold search of the makwargs["x"]imum likelihood procedure.
    # - kwargs['BETA']: the slope of the psychometric function.
    # - kwargs['GAMMA']: the lower limit of the psychometric function ekwargs["x"]pressed as a
    # proportion. In nAFC tasks kwargs['GAMMA'] is a single value (e.g., 0.5 for 2AFC).
    # In yes/no tasks it can be an array that contains values of false alarm
    # rates.
    # - LAMBDA: the subject's lapse rate ekwargs["x"]pressed as a proportion.

    ll = np.zeros([len(kwargs['alpha']), 1])

    # calculate the likelihood of each psychometric function
    for i in range(1, len(kwargs['alpha'])):
        # for j in range(1):
        ll[i, 0] = CalculateLikelihood(x = kwargs["x"], responses = kwargs["responses"],alpha= kwargs['alpha'][i],beta= kwargs['beta'], gamma = kwargs['gamma'] , _lambda=kwargs['_lambda'])
       

    # find the most likely psychometric function
    i, j = np.nonzero(ll == np.amax(ll, axis = 0))
    if (len(i) + len(j)) > 2:
        i = i[1]
        j = j[1]
    
    # calculate the level of the stimulus at p_target performance
    level = InvLogistic(p_target = kwargs["p_target"], alpha=kwargs['alpha'][i], beta =kwargs['beta'], gamma =kwargs['gamma'],_lambda= kwargs['_lambda'])
    FA = kwargs['gamma']
    # print(level, FA)
    return [level, FA]


# test
for i in range(1,25):
    print(f'Threshold Value no. {i}: ',FindThreshold(p_target= 0.809017, x=[2, 2.5, 4], responses=[0,1,0], alpha=np.arange(0.507, 0.577, 0.007),beta=i, gamma=0.5, _lambda=0.03)[0])
    

Threshold Value no. 1:  [0.98821187]
Threshold Value no. 2:  [0.74760594]
Threshold Value no. 3:  [0.66740396]
Threshold Value no. 4:  [0.62730297]
Threshold Value no. 5:  [0.60324237]
Threshold Value no. 6:  [0.58720198]
Threshold Value no. 7:  [0.57574455]
Threshold Value no. 8:  [0.56715148]
Threshold Value no. 9:  [0.56046799]
Threshold Value no. 10:  [0.55512119]
Threshold Value no. 11:  [0.55074653]
Threshold Value no. 12:  [0.54710099]
Threshold Value no. 13:  [0.5440163]
Threshold Value no. 14:  [0.54137228]
Threshold Value no. 15:  [0.53908079]
Threshold Value no. 16:  [0.53707574]
Threshold Value no. 17:  [0.53530658]
Threshold Value no. 18:  [0.53373399]
Threshold Value no. 19:  [0.53232694]
Threshold Value no. 20:  [0.53106059]
Threshold Value no. 21:  [0.52991485]
Threshold Value no. 22:  [0.52887327]
Threshold Value no. 23:  [0.52792226]
Threshold Value no. 24:  [0.52705049]


c:\users\anand\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:91: RuntimeWarning: divide by zero encountered in log
